<a href="https://colab.research.google.com/github/Priyo-prog/Deep-Learning-with-Tensorflow/blob/main/Natural%20Language%20Processing/NLP_Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Introduction

NLP has the goal of derieving information out of natural language (could be sequence text or speech).

Another common terms for NL problems is sequence to sequence problems (seq2seq).

## Check For GPU

In [167]:
!nvidia-smi

Sat Sep 28 05:34:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0              32W /  70W |    259MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Get Helper Functions

In [168]:
# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2024-09-28 05:34:56--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.3’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-09-28 05:34:56 (111 MB/s) - ‘helper_functions.py.3’ saved [10246/10246]



In [169]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Get a Text Dataset

Thedataset we are going to use is Kaggle's intoduction to NLP datset (testx sample of tweets labelled as disaster or not disaster) Binary Classification

In [170]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2024-09-28 05:34:56--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.207, 74.125.137.207, 142.250.101.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.3’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2024-09-28 05:34:56 (138 MB/s) - ‘nlp_getting_started.zip.3’ saved [607343/607343]



## Import Libraries

In [171]:
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

## Visualizing Text Dataset

In [172]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [173]:
train_df["text"][1]

'Forest fire near La Ronge Sask. Canada'

In [174]:
# Shuffle the training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [175]:
train_df["target"].value_counts()

,count
target,
0,4342
1,3271


In [176]:
# Let's visualize some random training samples
# create random indexes not higher than the total number of samples
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  # First underscore is to skip the index number
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n {text}\n")
  print("-----\n")

Target: 0 (not real disaster)
Text:
 if firefighters acted like cops they'd drive around shooting a flamethrower at burning buildings

-----

Target: 1 (real disaster)
Text:
 @dirk_trossen 

I've still got some of the snowstorm/hailstorm!

-----

Target: 0 (not real disaster)
Text:
 The last few days of summer are supposed to be the most fun so what's more fun then accidentally burning arm hair while playing w/ a lighter

-----

Target: 0 (not real disaster)
Text:
 A staged locomotive wreck what could possibly go wrong? http://t.co/Ei9x4H8tHm

-----

Target: 0 (not real disaster)
Text:
 @gregorysanders @USDOT &amp; the stat of high auto deaths applies to children in a vehicle. I guess they can out run lightrail better than adult

-----



## Split Data Into Training And Validation Sets

In [177]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    train_df_shuffled["text"].to_numpy(), train_df_shuffled["target"].to_numpy(),
    test_size=0.1, random_state=42)

In [178]:
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [179]:
# Check the first 10 sentences
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Change Text to Numbers

**Tokenization** - Straight mapping from token(a token could be aword or character) to number (can be modelled but quickly gets too big). Can be direct mapping like `0,1,2` or `A-1, B-2.... Z-26` or `one hot encoding` in that case it would be too large to handle

**Embedding** - Richer representation of relationships between tokens (can limit size + can be learned). This can update at each epochs of deep neural network learning.

In [180]:
# Use the Default Text Vectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create gropus of n-words
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None, # how long do you want your sequences to be
                                    pad_to_max_tokens=False)

## Average number of tokens in training tweets

In [181]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

## Setup Text Vectorization Variables

In [182]:
max_vocab_length = 10000 # max number of vocab to have in our vocabulary
max_length = 15 # max length our sequences will be(e.g. how many words from a tweet does a model see ?)
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [183]:
# Fit the text vectorizer to the training set
text_vectorizer.adapt(train_sentences)

In [184]:
# Create sample sentence and tokenize it
sample_sentence = "There's a flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [185]:
# choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original Sentence \n {random_sentence}\
      \n\nVectorized Version: ")
text_vectorizer([random_sentence])

Original Sentence 
 SEVERE WEATHER BULLETIN #6
TROPICAL CYCLONE WARNING: TYPHOON 'HANNA' 
Issued at 11:00 p.m. Thursday 06 August... http://t.co/FQV47OB8gE      

Vectorized Version: 


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 272,  229, 3134,  560, 1576,  582,  338,  494,    1,  886,   17,
        6369,  176, 1056, 1445]])>

In [186]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all the unique words in the vocabulary
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f"Number of words in vocab : {len(words_in_vocab)}")
print(f"5 most common words : {top_5_words}")
print(f"5 least common words : {bottom_5_words}")

Number of words in vocab : 10000
5 most common words : ['', '[UNK]', 'the', 'a', 'in']
5 least common words : ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


## Create Embedding Using Embedding Layer

To make our embedding we are going to use Tensorflow's embedding layer : https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The parameters we care most about our embedding layer:
* `input_dim` - The size of our vocabulary
* `output_dim` - the size of the output embedding vector, for example, a value of 100 would mean each tokens gets represented by vector 100 long

* `input_length` - Length of the sequences being passed to the embedding layer

In [187]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, #output_shape
                             input_length=max_length, # how long is each input
                             )
embedding

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


<Embedding name=embedding_3, built=False>

In [188]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original Text :\n{random_sentence}\
       \n\n Embedded version : ")

# Embded the random sentence (turn it into dense vector of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original Text :
Another White mass murderer. #Antioch  https://t.co/OWpd7vcFS6       

 Embedded version : 


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.03221258, -0.02920247, -0.03785907, ..., -0.04324174,
          0.01340542,  0.03988241],
        [-0.03779572,  0.00530333, -0.00439743, ...,  0.03890577,
          0.0055909 , -0.02086811],
        [ 0.04012254,  0.03291312,  0.04083129, ..., -0.03918055,
          0.04356335,  0.03994044],
        ...,
        [-0.04298794,  0.01016399,  0.02848453, ...,  0.04541465,
          0.00303783, -0.01270418],
        [-0.04298794,  0.01016399,  0.02848453, ...,  0.04541465,
          0.00303783, -0.01270418],
        [-0.04298794,  0.01016399,  0.02848453, ...,  0.04541465,
          0.00303783, -0.01270418]]], dtype=float32)>

In [189]:
# Check out single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-3.22125778e-02, -2.92024743e-02, -3.78590710e-02,  3.34181525e-02,
        -3.34629305e-02,  1.28687881e-02, -2.15715170e-03, -4.37811725e-02,
         4.69007976e-02, -1.27784163e-03,  4.18061018e-03, -2.08102539e-03,
         3.73672731e-02, -2.46952418e-02,  4.19625752e-02, -3.63409296e-02,
         7.56870583e-03,  4.88157906e-02, -7.13241100e-03, -2.09597237e-02,
        -2.30956562e-02,  4.74556424e-02,  3.01625021e-02, -3.21269631e-02,
        -1.28895417e-02, -4.22070175e-03, -3.91351059e-03, -8.74718279e-03,
        -3.51786502e-02,  2.01677792e-02, -2.16875076e-02, -9.77888703e-05,
        -4.49576154e-02,  4.32864763e-02, -4.22619581e-02,  4.88533489e-02,
        -1.08407363e-02, -3.92593965e-02, -3.87671813e-02, -3.60742584e-02,
         3.27082537e-02,  2.92192139e-02, -4.97804061e-02, -1.71415694e-02,
         4.27979119e-02, -3.63443010e-02,  5.61257452e-03, -4.09859531e-02,
         4.92412187e-02,  4.88906242e-0

## Modelling a dataset (running a set of experiments)

Now we've a got way to turn out text sequences into numbers it's time to start building a series of modelling experiments.

We'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline), this is from sklearn-map
* Model 1: Feed-Forward Neural Network (Dense Model)
* Model 2: LSTM model (RNN)
* Model 3: GRU Model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: Tensorflow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data

**Approach :**

* Create a model
* Build a model
* Fit a model
* Evaluate our model




## Model 0: Getting a baseline

As with all machine learning modelling experiments, it's important to create baseline model so you've got a benchmark for future experiments to build upon.

To crate our baseline, we'll use Sklearn's Multinomial Naive Bayes using TF-IDF Formula to convert our words to numbers.

**Note** : It's common use to non-DL algorithms as a baseline because of their speed and then later using DL to see if you improve upon them.

In [190]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create Tokenization and modelling pipeline
model_0 = Pipeline(
    steps=[("ifidf", TfidfVectorizer()), # convert numbers into words using tfidf
           ("clf", MultinomialNB()) # Model The text
           ])

# Fit The pipeline to training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('ifidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [191]:
## Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [192]:
train_df["target"].value_counts()

,count
target,
0,4342
1,3271


In [193]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

## Creating Evaluation Function for Model Experiments

In [194]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
   Calculates model accuracy, recall, precision and f1 score
   for a binary classification model
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1-score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [195]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [196]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save Tensorboard logs
SAVE_DIR = "model_logs"

## Create Model With Functional API

In [197]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string) # Inputs are 1-dimensional String
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x)  # Flatten the embedding output
outputs = layers.Dense(units=1, activation=tf.keras.activations.sigmoid)(x) # Create the output layer, want binary outputs so use sigmoid activation function
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [198]:
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_12 (InputLayer)          │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_7                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_3 (Embedding)              │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_4           │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [199]:
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_12 (InputLayer)          │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_7                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_3 (Embedding)              │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_4           │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [200]:
# compile the model
model_1.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [201]:
# fit the model
model_1_history = model_1.fit(x=train_sentences,
                               y=train_labels,
                               epochs=5,
                              validation_data=(val_sentences, val_labels),
                               callbacks=[create_tensorboard_callback(dir_name="SAVE_DIR",
                                                                      experiment_name="model_1_dense")])

Saving TensorBoard log files to: SAVE_DIR/model_1_dense/20240928-053457
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6332 - loss: 0.6520 - val_accuracy: 0.7612 - val_loss: 0.5344
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8058 - loss: 0.4665 - val_accuracy: 0.7822 - val_loss: 0.4743
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8569 - loss: 0.3597 - val_accuracy: 0.7874 - val_loss: 0.4590
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8903 - loss: 0.2904 - val_accuracy: 0.7953 - val_loss: 0.4648
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9142 - loss: 0.2351 - val_accuracy: 0.7900 - val_loss: 0.4839


## Check The Model Predictions

In [202]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7774 - loss: 0.5183


[0.4839259386062622, 0.7900262475013733]

In [203]:
# make some predictions and evaluate those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


(762, 1)

If we didn't put the GlobalAveragePooling1D for flatenning the layer then prediction would have come for all the tokens rather than the entire sample sentence

In [204]:
model_1_pred_probs[0]

array([0.34152928], dtype=float32)

In [205]:
# Look at The first 10 predictions
model_1_pred_probs[:10]

array([[0.34152928],
       [0.7611846 ],
       [0.9975067 ],
       [0.08340004],
       [0.12309837],
       [0.9287205 ],
       [0.91468847],
       [0.9934389 ],
       [0.9619784 ],
       [0.19622278]], dtype=float32)

## Compare Predictions With Labels

In [206]:
 # Convert model prediction probabilities to label format
 model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
 model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)>

## Classification result comparison of Labels and Predictions

In [207]:
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 79.00262467191601,
 'precision': 0.7980691136385264,
 'recall': 0.7900262467191601,
 'f1': 0.7861489606524888}

In [208]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

In [209]:
 baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Visualizing The Learned Embeddings

In [210]:
## Get the vocabulary from the text vectorization layer
word_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [212]:
# Get the weight matrix of the embedding layer
# (these are the numerical representations of each token in our training data, which have been learned for ~5 epochs)
embed_weights = model_1.get_layer("embedding_3").get_weights()[0]
embed_weights.shape # same size as vocab size and embedding_dim (output_dim of our embedding layer)

(10000, 128)

Now We've got the embedding matrix our model has learnt to represent our tokens, let's see how we can visualize it.

To do so tensorflow has a handy tool called projector:

And Tensorflow also has an incredible guide on word embeddings themselves :
http://projector.tensorflow.org/

https://www.tensorflow.org/text/guide/word_embeddings


In [213]:
import io

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [214]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent Neural Networks (RNN)

RNN's are useful for sequence data

The premise of a recurrent neural network is to use the  representation of previous input to aid the  representation of a later input.


**Resources**
- MIT 's sequence modelling lecture
https://www.youtube.com/watch?v=QvkQ1B3FBqA

- Chris Olah's intro to LSTMs:
https://colah.github.io/posts/2015-08-Understanding-LSTMs/

- Anderj Karpathy's the unreasonable effectiveness of recurrent neural networks :

https://karpathy.github.io/2015/05/21/rnn-effectiveness/

## Model 2: LSTM

LSTM = Long Short Term Memory (one of the most popular LSTM cells)

Our structure f an RNN looks like this:

```
Input (text) -> Tokenize -> Embedding -> Layers(RNN/Dense) -> Output (label probability)

````

In [215]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.LSTM(units=64, return_sequences=True)(x) # when you're stacking RNN cells together, you need to set return_sequences=True
print(x.shape)
x = layers.LSTM(units=64)(x)
print(x.shape)
x = layers.Dense(units=64, activation=tf.keras.activations.relu)(x)
outputs = layers.Dense(units=1, activation=tf.keras.activations.sigmoid)(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_lstm")

(None, 15, 128)
(None, 15, 64)
(None, 64)


In [216]:
# model summary
model_2.summary()

Model: "model_2_lstm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_13 (InputLayer)          │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_7                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_3 (Embedding)              │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_10 (LSTM)                       │ (None, 15, 64)              │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_11 (LSTM)                       │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,366,657 (5.21 MB)

 Trainable params: 1,366,657 (5.21 MB)

 Non-trainable params: 0 (0.00 B)

In [217]:
# compile the model
model_2.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [218]:
# Now fit the model
model_2.history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20240928-053536
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.8974 - loss: 0.2889 - val_accuracy: 0.7782 - val_loss: 0.6206
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9390 - loss: 0.1606 - val_accuracy: 0.7743 - val_loss: 0.7244
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9501 - loss: 0.1307 - val_accuracy: 0.7769 - val_loss: 0.6099
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9545 - loss: 0.1098 - val_accuracy: 0.7782 - val_loss: 0.8033
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9684 - loss: 0.0808 - val_accuracy: 0.7677 - val_loss: 1.0043


## Make Prediction With LSTM Model

In [219]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step


array([[5.6083631e-03],
       [8.8753366e-01],
       [9.9997079e-01],
       [2.7593926e-02],
       [1.6064121e-04],
       [9.9975580e-01],
       [8.9360893e-01],
       [9.9998128e-01],
       [9.9996483e-01],
       [6.1987263e-01]], dtype=float32)

## Convert the Model 2 Prediction to Labels

In [220]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [221]:
# Calculate model 2 results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 76.77165354330708,
 'precision': 0.769927879249698,
 'recall': 0.7677165354330708,
 'f1': 0.7652718344725522}

In [222]:
 baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 3 : GRU

Another popular and effective RNN component is the GRU or gated recurrent unit.

The GRU cells has similar features to LSTM cell but  has less parameters.

In [223]:
from tensorflow.keras import layers

inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.GRU(units=64, return_sequences=True)(x) # if you want to stack recurrent layers on top of each others, you need return_sequences=True
print(x.shape)
x = layers.LSTM(units=64, return_sequences=True)(x)
print(x.shape)
x = layers.GRU(units=64, return_sequences=True)(x)
print(x.shape)
x = layers.Dense(units=64, activation="relu")(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(units=1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")


(None, 15, 128)
(None, 15, 64)
(None, 15, 64)
(None, 15, 64)


In [224]:
# Get the summary
model_3.summary()

Model: "model_3_GRU"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_14 (InputLayer)          │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_7                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_3 (Embedding)              │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_8 (GRU)                          │ (None, 15, 64)              │          37,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_12 (LSTM)                       │ (None, 15, 64)              │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_9 (GRU)                          │ (None, 15, 64)              │          24,960 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 15, 64)              │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_5           │ (None, 64)                  │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,379,457 (5.26 MB)

 Trainable params: 1,379,457 (5.26 MB)

 Non-trainable params: 0 (0.00 B)

In [225]:
# compile the model
model_3.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# fit the model
model_3.fit(train_sentences,
            train_labels,
            epochs=5,
            validation_data=(val_sentences, val_labels),
            callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                   "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20240928-053550
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.9020 - loss: 0.2629 - val_accuracy: 0.7743 - val_loss: 1.0077
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9654 - loss: 0.0834 - val_accuracy: 0.7651 - val_loss: 0.9053
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9711 - loss: 0.0636 - val_accuracy: 0.7769 - val_loss: 1.0160
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9736 - loss: 0.0583 - val_accuracy: 0.7598 - val_loss: 1.3152
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9740 - loss: 0.0555 - val_accuracy: 0.7585 - val_loss: 1.9866


## Make Prediction With GRU Model

In [226]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step


array([[3.9026944e-04],
       [7.7610880e-01],
       [9.9999928e-01],
       [5.6347892e-02],
       [7.9302161e-08],
       [9.9999833e-01],
       [9.9785244e-01],
       [1.0000000e+00],
       [9.9999988e-01],
       [7.5599337e-01]], dtype=float32)

## Convert the Model 3 Prediction to Labels

In [227]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

## Model 4 : Bidirectional RNN Model

`Standard RNN`
In a typical RNN, the model processes sequential data by maintaining a hidden state that captures information from the past (previous time steps) as it moves from the beginning to the end of a sequence. However, it only considers the previous (past) context, which may not be ideal in tasks where the future context also plays an important role.

`Bidirectional RNN`
A Bidirectional RNN addresses this limitation by processing the sequence data in both directions:

Forward RNN: Processes the sequence from the first time step to the last (like a standard RNN).
Backward RNN: Processes the sequence from the last time step to the first (in reverse order).

In [228]:
# Build a Bidirectional RNN in Tensorflow
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
print(x.shape)
x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(units=1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidirectional")

(None, 15, 128)


In [229]:
model_4.summary()

Model: "model_4_bidirectional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_15 (InputLayer)          │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_7                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_3 (Embedding)              │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_13 (Bidirectional)     │ (None, 15, 128)             │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_14 (Bidirectional)     │ (None, 128)                 │          74,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,453,441 (5.54 MB)

 Trainable params: 1,453,441 (5.54 MB)

 Non-trainable params: 0 (0.00 B)

In [230]:
# compile the model 4
model_4.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20240928-053608
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.9559 - loss: 0.1779 - val_accuracy: 0.7598 - val_loss: 1.1303
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9774 - loss: 0.0500 - val_accuracy: 0.7572 - val_loss: 1.0664
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9782 - loss: 0.0486 - val_accuracy: 0.7756 - val_loss: 1.3078
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9778 - loss: 0.0461 - val_accuracy: 0.7756 - val_loss: 1.3371
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9799 - loss: 0.0398 - val_accuracy: 0.7585 - val_loss: 1.4953
